In [1]:
import os
import plotly.express as px
import pandas as pd
import numpy as np
import json

from utils import *
from train_eval import *
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

# Set seed
np.random.seed(42)

# Set working directory
os.chdir(r"..") # should be the git repo root directory, checking below:
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast"
dir_path = os.path.join(os.getcwd(), 'data', 'clean_data')
model_dir = os.path.join(os.getcwd(), 'models')



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: nikolaushouben (wattcast). Use `wandb login --relogin` to force relogin


Current working directory: c:\Users\nik\Desktop\Berkeley_Projects\WattCast


In [2]:
def train_eval_light():

    wandb.init(project="WattCast_tuning")
    wandb.config.update(config_run)
    config = wandb.config

    print("Getting data...")

    pipeline, ts_train_piped, ts_val_piped, ts_test_piped, ts_train_weather_piped, ts_val_weather_piped, ts_test_weather_piped, trg_train_inversed, trg_val_inversed, trg_test_inversed = data_pipeline(config)

    print("Getting model instance...")
    model = get_model(config)
    model, runtime = train_models([model], ts_train_piped, ts_train_weather_piped, ts_val_piped, ts_val_weather_piped)

    print("Evaluating model...")
    predictions, score = predict_testset(model[0], 
                                  ts_test_piped[config.longest_ts_test_idx], 
                                  ts_test_weather_piped[config.longest_ts_test_idx],
                                  config.n_lags, config.n_ahead, config.eval_stride, pipeline,
                                  )


    print("Plotting predictions...")
    df_compare = pd.concat([trg_test_inversed.pd_dataframe(), predictions], axis=1).dropna()
    df_compare.columns = ['target', 'prediction']
    fig = px.line(df_compare, title='Predictions vs. Test Set')

    wandb.log({'eval_loss': score})
    wandb.log({'predictions': fig})
    wandb.finish()



In [3]:
# See what keys are in the h5py data file
locations_dict, resolutions_dict = get_hdf_keys(dir_path)

print("Locations: ", locations_dict)
print("Resolutions: ", resolutions_dict)

Locations:  {'1_county.h5': ['Los_Angeles', 'New_York', 'Sacramento'], '2_town.h5': ['town_0', 'town_1', 'town_2'], '3_village.h5': ['village_0', 'village_1', 'village_2'], '4_neighborhood.h5': ['neighborhood_0', 'neighborhood_1', 'neighborhood_2'], '5_building.h5': ['building_0', 'building_1', 'building_2']}
Resolutions:  {'1_county.h5': ['60min'], '2_town.h5': ['15min', '60min'], '3_village.h5': ['15min', '60min'], '4_neighborhood.h5': ['60min'], '5_building.h5': ['60min']}


In [ ]:
for scale, locations in locations_dict.items():
    if scale == '4_neighborhood.h5':
        for location in locations:
            df_train = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/train_target')
            df_val = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/val_target')
            df_test = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/test_target')
            fig = px.line(df_train, title=f'{scale}: {location}')
            fig.add_trace(px.line(df_val, title='Validation Set').data[0])
            fig.add_trace(px.line(df_test, title='Test Set').data[0])
            fig.show()



In [4]:
# run parameters

sweeps = 20

scale_location_pairs = (
    # ('1_county', 'Sacramento'),
    # ('1_county', 'New_York'),
    #('2_town', 'town_0'),
    # ('2_town', 'town_1'),
    # ('2_town', 'town_2'),
    # ('3_village', 'village_1'),
    # ('3_village', 'village_2'),
    #('3_village', 'village_0'),
    ('4_neighborhood', 'neighborhood_0'),
    # ('4_neighborhood', 'neighborhood_1'),
    # ('4_neighborhood', 'neighborhood_2'),
    # ('5_building', 'building_0'),
    # ('5_building', 'building_1'),
    # ('5_building', 'building_2'),
      )


models = [
        # 'rf',
        # 'xgb', 
        # 'gru', 
        # 'lgbm',  
        # 'nbeats',
        #'transformer',
        'tft'
        ]

for scale, location in scale_location_pairs:

    for model in models:
        # place holder initialization of config file (will be updated in train_eval_light()
        config_run = {
            'spatial_scale': scale,
            'temp_resolution': 60,
            'location': location,
            'model': model,
            'horizon_in_hours': 24,
            'lookback_in_hours': 24,
            'boxcox': True,
            'liklihood': None,
            'weather': True,
            'holiday': True,
            'datetime_encodings': False,
        }
        
        with open(f'sweep_configurations/config_sweep_{model}.json', 'r') as fp:
            sweep_config = json.load(fp)                  

        sweep_config['name'] = model + 'sweep' + config_run['spatial_scale'] + '_' + config_run['location'] + '_' + str(config_run['temp_resolution'])

        sweep_id = wandb.sweep(sweep_config, project="WattCast_tuning")
        wandb.agent(sweep_id, train_eval_light, count=sweeps)


Create sweep with ID: py97pr7v
Sweep URL: https://wandb.ai/wattcast/Wattcast_tuning/sweeps/py97pr7v


wandb: Agent Starting Run: zisp9za7 with config:
wandb: 	batch_size: 128
wandb: 	datetime_encodings: 0
wandb: 	dropout: 0.1
wandb: 	hidden_dim: 512
wandb: 	lr: 0.001
wandb: 	n_rnn_layers: 3
wandb: 	num_attention_heads: 8
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Config item 'datetime_encodings' was locked by 'sweep' (ignored update).


Getting data...
Getting model instance...
Training TFTModel


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                              | Type                             | Params
----------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0     
1  | val_metrics                       | MetricCollection                 | 0     
2  | input_embeddings                  | _MultiEmbedding                  | 0     
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 0     
4  | encoder_vsn                       | _VariableSelectionNetwork        | 37.5 K
5  | decoder_vsn                       | _VariableSelectionNetwork        | 25.0 K
6  | static_context_grn                | _GatedResidualNetwork            | 1.1 M 
7  | static_context_hidden_encoder_grn | _GatedResid

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]